In [1]:
import pandas as pd
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [18]:
import os,random,math
TRAINING_DIR="propaganda_dataset_v2"
files=os.listdir(TRAINING_DIR)
trainingdf= pd.read_csv(os.path.join(TRAINING_DIR,files[1]),sep = '\t')
testingdf= pd.read_csv(os.path.join(TRAINING_DIR,files[2]),sep = '\t')


In [19]:
def convert_text(sentences):
    start_sent = sentences.replace("<BOS>","")
    end_sent =start_sent.replace("<EOS>","")
    return end_sent

In [20]:
testingdf["tagged_in_context"]=testingdf["tagged_in_context"].map(convert_text)
trainingdf["tagged_in_context"]=trainingdf["tagged_in_context"].map(convert_text)

In [21]:
binaryclass_traindf=trainingdf
binaryclass_testdf=testingdf

binaryclass_traindf

,label,tagged_in_context
0,not_propaganda,"No, he will not be confirmed."
1,not_propaganda,This declassification effort won’t make thing...
2,flag_waving,The Obama administration misled the American ...
3,not_propaganda,“It looks like we’re capturing the demise of t...
4,not_propaganda,"Location: Westerville, Ohio"
...,...,...
2409,not_propaganda,We support and appreciate your business.”
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...
2411,not_propaganda,What has been done: there has been work on for...
2412,not_propaganda,This is the law of gradualness not the gradua...


In [22]:
def convert_labels(label):
    return 1 if label=="not_propaganda" else 0
binaryclass_testdf["label"]= binaryclass_testdf["label"].map(convert_labels)
binaryclass_traindf["label"]=binaryclass_traindf["label"].map(convert_labels)
binaryclass_traindf

,label,tagged_in_context
0,1,"No, he will not be confirmed."
1,1,This declassification effort won’t make thing...
2,0,The Obama administration misled the American ...
3,1,“It looks like we’re capturing the demise of t...
4,1,"Location: Westerville, Ohio"
...,...,...
2409,1,We support and appreciate your business.”
2410,1,International Atomic Energy Agency (IAEA) Dire...
2411,1,What has been done: there has been work on for...
2412,1,This is the law of gradualness not the gradua...


In [23]:
from nltk import word_tokenize
binaryclass_testdf["tagged_in_context"]= binaryclass_testdf["tagged_in_context"].apply(word_tokenize)
binaryclass_traindf["tagged_in_context"]=binaryclass_traindf["tagged_in_context"].apply(word_tokenize)

In [24]:
binaryclass_traindf

,label,tagged_in_context
0,1,"[No, ,, he, will, not, be, confirmed, .]"
1,1,"[This, declassification, effort, won, ’, t, ma..."
2,0,"[The, Obama, administration, misled, the, Amer..."
3,1,"[“, It, looks, like, we, ’, re, capturing, the..."
4,1,"[Location, :, Westerville, ,, Ohio]"
...,...,...
2409,1,"[We, support, and, appreciate, your, business,..."
2410,1,"[International, Atomic, Energy, Agency, (, IAE..."
2411,1,"[What, has, been, done, :, there, has, been, w..."
2412,1,"[This, is, the, law, of, gradualness, not, the..."


In [25]:
binaryclass_traindf

,label,tagged_in_context
0,1,"[No, ,, he, will, not, be, confirmed, .]"
1,1,"[This, declassification, effort, won, ’, t, ma..."
2,0,"[The, Obama, administration, misled, the, Amer..."
3,1,"[“, It, looks, like, we, ’, re, capturing, the..."
4,1,"[Location, :, Westerville, ,, Ohio]"
...,...,...
2409,1,"[We, support, and, appreciate, your, business,..."
2410,1,"[International, Atomic, Energy, Agency, (, IAE..."
2411,1,"[What, has, been, done, :, there, has, been, w..."
2412,1,"[This, is, the, law, of, gradualness, not, the..."


In [11]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))

two_class_testdf["tagged_in_context"]=two_class_testdf["tagged_in_context"].apply(str)
two_class_traindf["tagged_in_context"]=two_class_traindf["tagged_in_context"].apply(str)
x_train= vectorizer.fit_transform(two_class_traindf["tagged_in_context"])
x_test = vectorizer.transform(two_class_testdf["tagged_in_context"])
y_train = two_class_traindf["label"]
y_test = two_class_testdf["label"]
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
x_train.shape
x_test.shape

(580, 34151)

In [12]:
y_pred = mnb.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred, average='macro'))
print("Recall: ", recall_score(y_test, y_pred, average='macro'))
print("F1 Score: ", f1_score(y_test, y_pred, average='macro'))

Accuracy:  0.6637931034482759
Precision:  0.6898915816326531
Recall:  0.6701854034937306
F1 Score:  0.6567641140963427
